In [33]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# <span style="font-family:serif; color:blue"> Import Libraries</span>

In [ ]:
import numpy as np
import os

# visualization libraries
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib.image import imread

# tensorflow libraries
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, Model, load_model
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras import optimizers
from keras.preprocessing import image

# model evaluation libraries
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

# load image using the old keras version
img = image.load_img('path/to/image.jpg', target_size=(224, 224))

# convert image to numpy array
img_array = image.img_to_array(img)

# expand dimensions to match the shape expected by the model
img_array = np.expand_dims(img_array, axis=0)

# normalize pixel values
img_array /= 255.0


### <span style="font-family:serif; color:blue"> Let's define each directories</span>

In [ ]:
base_dir= "/kaggle/input/files1/Malaria Cells"
training_dir= "/kaggle/input/files1/Malaria Cells/training_set"
testing_dir= "/kaggle/input/files1/Malaria Cells/testing_set"

print("inside base dir :", os.listdir(base_dir))
print("\ntraining_set :", os.listdir(training_dir))
print("\ntesting_set :", os.listdir(testing_dir))

In [ ]:
train_Uninfected= "/kaggle/input/files1/Malaria Cells/training_set/Uninfected"
train_Parasitized= "/kaggle/input/files1/Malaria Cells/training_set/Parasitized"

test_Uninfected= "/kaggle/input/files1/Malaria Cells/testing_set/Uninfected"
test_Parasitized= "/kaggle/input/files1/Malaria Cells/testing_set/Parasitized"

print("train_Uninfected:", os.listdir(train_Uninfected)[:5])
print("\ntrain_Parasitized:", os.listdir(train_Parasitized)[:5])
print("\ntest_Uninfected:", os.listdir(test_Uninfected)[:5])
print("\ntest_Parasitized:", os.listdir(test_Parasitized)[:5])

### <span style="font-family:serif; color:blue"> Let's find out the total number of Uninfected and Parasitized images in the train and test directories:</span>

In [ ]:
print("Total train Uninfected cell images :", len(os.listdir(train_Uninfected)))
print("Total train Parasitized cell images :", len(os.listdir(train_Parasitized)))
print("\nTotal test Uninfected cell images :", len(os.listdir(test_Uninfected)))
print("Total test Parasitized cell images :", len(os.listdir(test_Parasitized)))

### <span style="font-family:serif; color:blue"> Now let's take a look at a few pictures of what the uninfected and parasitized cells look like.</span>

In [ ]:
nrows = 4
ncols = 4

pic_index = 0 

fig = plt.gcf()
fig.set_size_inches(ncols*4, nrows*4)

pic_index+=8

parasitized_cell = [os.path.join(train_Parasitized, image) for image in os.listdir(train_Parasitized)[pic_index-8:pic_index]]

uninfected_cell = [os.path.join(train_Uninfected, image) for image in os.listdir(train_Uninfected)[pic_index-8:pic_index]]

for i, image_path in enumerate(parasitized_cell+uninfected_cell):

  sp = plt.subplot(nrows, ncols, i + 1)
  sp.axis('Off') 

  img = mpimg.imread(image_path)
  plt.imshow(img)

plt.show()

## <span style="font-family:serif; color:blue"> Generating Data using augmentation</span>

In [ ]:
# generating training set
print("training data :")
train_datagen= ImageDataGenerator(rescale=1/255, zoom_range=0.3, rotation_range=50,
                                   width_shift_range=0.2, height_shift_range=0.2, shear_range=0.2, 
                                   horizontal_flip=True, fill_mode='nearest')

train_data= train_datagen.flow_from_directory(training_dir,
                                              target_size= (100,100),
                                              batch_size=20,
                                              class_mode= "binary",
                                              shuffle= True)


# generating validation set
print("\nvalidation data :")
val_datagen= ImageDataGenerator(rescale=1/255) # data augmentation is applied only to training data not to validation & test data

val_data= val_datagen.flow_from_directory(testing_dir,
                                          target_size= (100,100),
                                          batch_size=20,
                                          class_mode= "binary",
                                          shuffle= False)

# generating test set
print("\ntest data :")
test_datagen= ImageDataGenerator(rescale=1/255)

test_data= test_datagen.flow_from_directory(testing_dir,
                                            target_size= (100,100),
                                            batch_size=20,
                                            class_mode= "binary",
                                            shuffle= False)

### <span style="font-family:serif; color:blue"> Lets's see what labels are assigned to each class</span>

In [ ]:
train_data.class_indices

# <span style="font-family:serif; color:blue"> Model Building - CNN</span>

In [ ]:
model_1= Sequential()

model_1.add(Conv2D(filters= 16, kernel_size=(3,3), activation="relu", input_shape=(100, 100, 3)))
model_1.add(MaxPooling2D(pool_size=(2,2)))

model_1.add(Conv2D(filters= 32, kernel_size=(3,3), activation="relu"))
model_1.add(MaxPooling2D(pool_size=(2,2)))

model_1.add(Conv2D(filters= 64, kernel_size=(3,3), activation="relu"))
model_1.add(MaxPooling2D(pool_size=(2,2)))

model_1.add(Conv2D(filters= 128, kernel_size=(3,3), activation="relu"))
model_1.add(MaxPooling2D(pool_size=(2,2)))

model_1.add(Flatten())
model_1.add(Dense(units= 512, activation='relu'))
model_1.add(Dropout(0.2))

model_1.add(Dense(units=1, activation="sigmoid"))

model_1.summary()

In [ ]:
# model compiling
model_1.compile(loss= "binary_crossentropy", optimizer=optimizers.RMSprop(lr=1e-4), metrics=["accuracy"])

# model fitting
history= model_1.fit_generator(train_data,
                               steps_per_epoch= train_data.samples//train_data.batch_size,
                               validation_data= val_data,
                               validation_steps= val_data.samples//val_data.batch_size,
                               epochs=20, verbose= 2)

# <span style="font-family:serif; color:blue"> Model Evaluation</span>

In [ ]:
history.history.keys()

In [ ]:
epochs= range(len(history.history["accuracy"]))
# accuracy plot
plt.plot(epochs, history.history["accuracy"])
plt.plot(epochs, history.history["val_accuracy"])
plt.xlabel("epoch")
plt.ylabel("accuracy")
plt.title("CNN Model Accuracy")
plt.legend(["train", "validation"])
plt.show()

# loss plot
plt.plot(epochs, history.history["loss"])
plt.plot(epochs, history.history["val_loss"])
plt.xlabel("epoch")
plt.ylabel("loss")
plt.title("CNN Model Loss")
plt.legend(["train", "validation"])
plt.show()

# <span style="font-family:serif; color:blue"> Using Transfer Learning- InceptionV3 Model</span>

In [ ]:
# inceptionV3 model, with include_top= False we are not using fully connected layer of the inceptionV3 model, instead we
#  will create our own Fully Connected and Output Layer according to our training data
inception= InceptionV3(input_shape= (100, 100,3), include_top= False, weights="imagenet")

# Since we are creating our own fully connected layer we need output of the last inception model layer and flatten them 
last_output= inception.layers[-1].output

# Flattening the last output
last_output= Flatten()(last_output)

# Our pretrained model
inception_model= Model(inception.input, last_output)

### *Creating Fully Connected Layer*

In [ ]:
# layer 1
x= Dense(units=512, activation="relu")(last_output)
x=Dropout(0.2)(x)

# layer 2
x= Dense(units=128, activation="relu")(x)
x=Dropout(0.2)(x)

# output layer
x= Dense(units=1, activation="sigmoid")(x)

# final model
model= Model(inception_model.input, x)

model.compile(loss="binary_crossentropy", optimizer=optimizers.RMSprop(lr=1e-4), metrics=["accuracy"])
model.summary()

### *Model fitting*

In [ ]:
# Since the layers of InceptionV3 model are already trained, we don't want them to be trained again. 
# So we will freeze these layers
for layer in inception_model.layers:
    layer.trainable= False

# fitting only Fully Connected layers    
history= model.fit(train_data, 
                   steps_per_epoch= train_data.samples//train_data.batch_size,
                   validation_data= val_data,
                   validation_steps= val_data.samples//val_data.batch_size,
                   epochs=20, verbose= 2)

## Model Evaluation

In [ ]:
epochs= range(len(history.history["accuracy"]))
# accuracy plot
plt.plot(epochs, history.history["accuracy"])
plt.plot(epochs, history.history["val_accuracy"])
plt.xlabel("epoch")
plt.ylabel("accuracy")
plt.title("InceptionV3 Model Accuracy")
plt.legend(["train", "validation"])
plt.show()

# loss plot
plt.plot(epochs, history.history["loss"])
plt.plot(epochs, history.history["val_loss"])
plt.xlabel("epoch")
plt.ylabel("loss")
plt.title("InceptionV3 Model Loss")
plt.legend(["train", "validation"])
plt.show()

### *Checking model performance through test data*

In [ ]:
prediction= model.predict_generator(test_data, steps=np.ceil(val_data.samples/val_data.batch_size), verbose=2)
prediction= prediction > 0.5
prediction

In [ ]:
cm= confusion_matrix(test_data.classes, prediction)
plot_confusion_matrix(cm, figsize=(5,5))

print(accuracy_score(test_data.classes, prediction))
print(classification_report(test_data.classes, prediction))

### *Finally let's predict the image present in the single_prediction directory with our second model*

In [ ]:
test_image= "/kaggle/input/files1/Malaria Cells/single_prediction/Parasitised.png"
test_image= load_img(test_image,target_size=(100, 100))
print("The following test image is of Parasitised Cell with label '0'")
test_image

In [ ]:
test_image= img_to_array(test_image)
test_image = np.expand_dims(test_image, axis=0)
print(test_image.shape)

In [ ]:
model.predict(test_image)

### Saving Model

In [ ]:
model.save("malaria2.h5") 

In [ ]:
# saving 1st model
model_1.save("malaria_cnn.h5")